
# 🚀 Experimento Completo: Detecção de Viés no TwiBot-22

Este notebook executa a pipeline completa de detecção de viés em **escala real**, conforme o artigo[cite: 1]. Ele é otimizado para um ambiente de alta performance (Colab Pro com A100/160GB RAM).

**Objetivo:** Comparar a **Heurística Corrigida (Enhanced Louvain)**  com o **Louvain Padrão** no dataset TwiBot-22 completo.

## ⚙️ Passo 0: Configuração do Ambiente

In [1]:
import os

# ‼️ ATENÇÃO: Confirme se este é o caminho para a pasta raiz do seu projeto
project_path_on_drive = '..'

os.chdir(project_path_on_drive)
print(f"✅ Diretório de trabalho alterado para: {os.getcwd()}")

✅ Diretório de trabalho alterado para: c:\Users\axlsa\Documents\bias-aware-community-detection


In [2]:
print("📦 Instalando dependências...")

# Passo 0: Atualizar pip
print("   Atualizando pip...")
!pip install --upgrade pip -q

# Passo 1: Instalar PyTorch (com CUDA) usando o URL especial
print("   Instalando PyTorch (CUDA)...")
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu130

# Passo 2: Instalar as outras bibliotecas (do repositório padrão)
print("   Instalando networkx, pandas, transformers, etc...")
!pip install ipywidgets scikit-learn networkx python-louvain pandas tqdm psutil transformers[torch] matplotlib seaborn -q

print("✅ Dependências instaladas!")

📦 Instalando dependências...
   Atualizando pip...
   Instalando PyTorch (CUDA)...



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
c:\Users\axlsa\Documents\bias-aware-community-detection\.venv\Scripts\python.exe -m pip install --upgrade pip -q


Looking in indexes: https://download.pytorch.org/whl/cu130
   Instalando networkx, pandas, transformers, etc...



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Dependências instaladas!



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
print("🔧 Importando módulos e configurando ambiente...")

import sys
import time
import json
import torch
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import community.community_louvain as louvain
import warnings
import random
from collections import defaultdict

# Adicionar 'src' ao sys.path para imports
project_dir_to_add = os.getcwd()
if project_dir_to_add not in sys.path:
    sys.path.append(project_dir_to_add)

# Importar nossos módulos
try:
    from src.config import Config
    from src.data_utils import TwiBotDataLoader
    from src.bias_calculator import BiasCalculator
    from src.heuristic import EnhancedLouvainWithBias # A versão CORRIGIDA
    from src.evaluation import ComprehensiveEvaluator
except ImportError as e:
    print(f"❌ ERRO: Não foi possível importar os módulos. Verifique se os arquivos .py estão corretos e o __init__.py existe.")
    print(f"Detalhe: {e}")
    raise e

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(Config.RANDOM_STATE)
random.seed(Config.RANDOM_STATE)
cfg = Config()

# --- Verificação de Hardware (Ambiente Local Windows 11) ---
print("\n--- Verificação de Hardware (Local) ---")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ GPU NVIDIA (CUDA) NÃO DETECTADA. Usando CPU (será lento).")

# cfg foi definido na célula anterior (cfg = Config())
print(f"✅ CPUs: {os.cpu_count()} (Config.NUM_WORKERS={cfg.NUM_WORKERS})")
print("✅ Ambiente local pronto!")

🔧 Importando módulos e configurando ambiente...

--- Verificação de Hardware (Local) ---
✅ GPU: NVIDIA GeForce RTX 3060
   VRAM: 12.9 GB
✅ CPUs: 12 (Config.NUM_WORKERS=12)
✅ Ambiente local pronto!


## 📊 Passo 2: Carregar Grafo Completo (Full Scale)

In [4]:
print("📊 Carregando/Construindo Grafo (Full Scale)...")
start_load = time.time()

data_loader = TwiBotDataLoader()

# Garantir que os diretórios de cache existem
cfg.create_dirs()

# Passar max_nodes=None para carregar o grafo inteiro
G, bot_labels = data_loader.load_and_build_graph(max_nodes=None)

load_time = time.time() - start_load
print(f"   ↳ Tempo total para carregar/construir grafo: {load_time:.2f}s")
print(f"\n📈 Grafo Carregado: {G.number_of_nodes():,} nós, {G.number_of_edges():,} arestas")
print(f"🎯 Bots Identificados: {sum(bot_labels.values()):,} ({sum(bot_labels.values())/len(bot_labels):.1%})")

📊 Carregando/Construindo Grafo (Full Scale)...
✅ Diretório criado: c:\Users\axlsa\Documents\bias-aware-community-detection\processed_data
✅ Diretório criado: c:\Users\axlsa\Documents\bias-aware-community-detection\processed_data
📊 Fase 1: Carregando/Construindo Grafo (NetworkX)...
   Início Carga Grafo RAM Usada: 763.8 MB
   Arquivos não encontrados. Construindo grafo do zero...
   ✅ 1,000,000 labels carregados.
   🔗 Carregando edge.csv em chunks...


1702it [33:24,  1.18s/it]


   Grafo inicial: 1,000,000 nós, 3,301,025 arestas.
   Encontrando maior componente conectado...
   📊 Grafo final (maior CC): 693,759 nós, 3,301,024 arestas.
   Após construir grafo RAM Usada: 1,743.7 MB

   💾 Salvando grafo processado e labels para uso futuro...
   ✅ Arquivos salvos.
   ↳ Tempo total para carregar/construir grafo: 2036.53s

📈 Grafo Carregado: 693,759 nós, 3,301,024 arestas
🎯 Bots Identificados: 81,431 (11.7%)


## 🧠 Passo 3: Calcular Scores de Viés (Full Scale)

Esta é a etapa computacionalmente mais intensiva. Ela irá:
1.  Ler os arquivos de `/tmp/` (rápido).
2.  Processar **todos** os tweets relevantes (sem limite artificial).
3.  Usar todos os seus `NUM_WORKERS` e a A100.

**Esta célula levará um tempo considerável (talvez horas), mas a barra de progresso `tqdm` mostrará o status.**

In [5]:
from collections import defaultdict

print("🧠 Calculando/Carregando Scores de Viés (Full Scale)...")
start_bias = time.time()

bias_calculator = BiasCalculator()

# Esta função agora está configurada para ler de /tmp/ e sem TWEET_LIMIT_PER_WORKER
bias_scores = bias_calculator.get_or_calculate_bias_scores(set(G.nodes()))

bias_time = time.time() - start_bias
print(f"   ↳ Tempo total para calcular/carregar viés: {bias_time:.2f}s")

# Análise exploratória dos scores reais
if bias_scores:
    bias_values = [s for s in bias_scores.values() if s != 0.0] # Ignorar nós com 0.0 (sem tweets)
    if bias_values:
        print(f"\n📊 Estatísticas do Viés (Nós com tweets: {len(bias_values):,}):")
        print(f"   Média={np.mean(bias_values):.3f}, Std={np.std(bias_values):.3f}")

        plt.figure(figsize=(10, 4))
        plt.hist(bias_values, bins=50, alpha=0.7, color='blue')
        plt.title('Distribuição dos Scores de Viés (Dataset Completo)')
        plt.xlabel('Score de Viés (-1 Negativo, 1 Positivo)')
        plt.ylabel('Frequência')
        plt.grid(True, alpha=0.3)
        plt.show()
    else:
        print("⚠️ Dicionário de viés foi criado, mas está vazio ou só contém zeros.")
else:
    print("⚠️ Nenhum score de viés foi calculado ou carregado.")

🧠 Calculando/Carregando Scores de Viés (Full Scale)...

🧠 Fase 2: Calculando/Carregando Scores de Viés...
   Início Viés RAM Usada: 1,670.1 MB
💾 Verificando se o arquivo final 'c:\Users\axlsa\Documents\bias-aware-community-detection\processed_data\bias_scores.json' já existe...
   Arquivo final não encontrado. Calculando...

--- Iniciando cálculo de scores de viés (Single-Pass Paralelo) ---
--- ATENÇÃO WINDOWS: Lendo do caminho original c:\Users\axlsa\Documents\bias-aware-community-detection\data\TwiBot22 ---
   (Caminho C:\tmp\tweet_data não encontrado)

--- Passagem Única: Processando 2 arquivos em paralelo (12 workers) ---
   Iniciando 2 workers. Acompanhe o progresso:


   Progresso Workers: 100%|██████████| 2/2 [59:03<00:00, 1771.63s/it]



📊 Processamento paralelo concluído em 3543.89 segundos.
   Após processamento paralelo: RAM Usada: 19.4 MB

⚙️ Agregando resultados dos workers...
   ↳ Total de tweets processados: 0
   ✅ Agregação concluída em 3.05s para 0 usuários.
   Após agregação: RAM Usada: 868.6 MB

⚙️ Calculando scores médios de viés por usuário...

⚙️ Garantindo scores...
   ↳ 693,759 nós sem tweets receberam score 0.0.

💾 Salvando scores finais em 'c:\Users\axlsa\Documents\bias-aware-community-detection\processed_data\bias_scores.json'...
   ✅ Scores finais salvos.

✅ Cálculo de viés (Completo) concluído.
   Final: RAM Usada: 886.0 MB
   ↳ Tempo total para calcular/carregar viés: 3549.23s
⚠️ Dicionário de viés foi criado, mas está vazio ou só contém zeros.


## 🎯 Passo 4: Executar Heurística Corrigida (Enhanced Louvain)

Executamos o algoritmo principal do artigo (a heurística corrigida) , usando `alpha=0.5` [cite: 18, 217] para balancear estrutura e viés.

In [6]:
print(f"\n🎯 Executando Enhanced Louvain com Viés (α={cfg.ALPHA})...")
start_enhanced = time.time()

# Usar a classe HEURISTIC.PY corrigida
detector = EnhancedLouvainWithBias(alpha=cfg.ALPHA, verbose=True)

# O artigo foca em partição binária (k=2) [cite: 68-71, 99-106]
detector.fit(G, bias_scores, num_communities=2)

communities_enhanced = detector.get_communities()
enhanced_time = time.time() - start_enhanced

print(f"\n avaliando resultados da Heurística...")
metrics_enhanced = ComprehensiveEvaluator.evaluate_communities(
    G, communities_enhanced, bias_scores, bot_labels
)
metrics_enhanced['runtime'] = enhanced_time

print(f"✅ Heurística concluída em {enhanced_time:.2f}s")


🎯 Executando Enhanced Louvain com Viés (α=0.5)...
🎯 Executando Enhanced Louvain (α=0.5)...
   Fase 1: Executando Louvain padrão para partição inicial...


KeyboardInterrupt: 

## ⚖️ Passo 5: Executar Baseline (Louvain Padrão)

Comparamos com o algoritmo de Louvain padrão, que otimiza apenas a modularidade (estrutura)[cite: 50].

In [ ]:
print("\n⚖️ Executando Louvain Padrão (Baseline)...")
start_louvain = time.time()

communities_louvain = louvain.best_partition(G, random_state=cfg.RANDOM_STATE)

louvain_time = time.time() - start_louvain

print(f"... Louvain Padrão encontrou {len(set(communities_louvain.values()))} comunidades.")
print("   Avaliando resultados do Louvain Padrão...")

metrics_louvain = ComprehensiveEvaluator.evaluate_communities(
    G, communities_louvain, bias_scores, bot_labels
)
metrics_louvain['runtime'] = louvain_time

print(f"✅ Louvain Padrão concluído em {louvain_time:.2f}s")

## 📈 Passo 6: Resultados Finais e Comparação

Aqui vemos se o nosso método (Enhanced) supera o Baseline (Padrão) nas métricas-chave do artigo: **Separação de Viés** e **Pureza de Viés** [cite: 136-138].

In [ ]:
print("\n" + "=" * 60)
print("📊 RESULTADOS FINAIS (Full Scale TwiBot-22)")
print("=" * 60)

ComprehensiveEvaluator.print_comparison(
    metrics_enhanced,
    metrics_louvain,
    "Enhanced Louvain (α=0.5)",
    "Louvain Padrão (α=0.0)"
)

print("\n--- Métricas Detalhadas ---")
df_results = pd.DataFrame({
    "Enhanced Louvain (α=0.5)": metrics_enhanced,
    "Louvain Padrão (α=0.0)": metrics_louvain
}).T

print(df_results[['modularity', 'bias_separation', 'bias_purity', 'bot_concentration_max', 'num_communities', 'runtime']].to_markdown(floatfmt=".4f"))

## 🔍 Passo 7: Análise Detalhada das Comunidades

Vamos inspecionar as duas comunidades que nosso método encontrou.

In [ ]:
print("\n🔍 Análise Detalhada (Enhanced Louvain)")

comm_nodes_map = defaultdict(list)
for node, comm in communities_enhanced.items():
    comm_nodes_map[comm].append(node)

for comm_id in sorted(comm_nodes_map.keys()):
    comm_nodes = comm_nodes_map[comm_id]
    comm_biases = [bias_scores.get(node, 0) for node in comm_nodes]
    comm_bots = [bot_labels.get(node, False) for node in comm_nodes]

    print(f"\n🏷️  Comunidade {comm_id}:")
    print(f"   • {len(comm_nodes):,} nós ({len(comm_nodes) / G.number_of_nodes():.1%})")
    print(f"   • Viés médio: {np.mean(comm_biases):.3f} (±{np.std(comm_biases):.3f})")
    print(f"   • Bots: {sum(comm_bots):,}/{len(comm_bots):,} ({sum(comm_bots)/len(comm_bots):.1%})")

print("\n" + "=" * 60)
print("🎉 EXPERIMENTO COMPLETO CONCLUÍDO!")
print("=" * 60)